In [1]:
#Контакты - danila2012r@yandex.ru
#tg - @aggle_flomaster

In [2]:
from matplotlib.pyplot import *
from matplotlib.animation import * 
from numpy import *
import warnings
warnings.filterwarnings("ignore")

In [3]:
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'qt') 

# Начальные данные и параметры задачи

In [4]:

t_0 = 0
t_1=50       #Временной отрезок [0;50]
x_0 = 3        #Начальные данные
y_0 = -4.
v_x_0 = -0.8 #Скорость
v_y_0 = -0.6
g = 9.81
L = 5.0
q=1/L
T=10
mass = 1.0
alpha = (1 + 1j)/2; 
M = 10000
tau = (t_1 - t_0)/M   #Шаг по времени
t = linspace(t_0,t_1,M + 1) #Сетка

In [5]:
def gravity(t):
    return 9.81 + 0.05*np.sin(2*np.pi*t)

# Правая часть

In [6]:
def f(u,g,mass,l):
    f = zeros(5)
    f[0] = u[2]
    f[1] = u[3]
    f[2] = -q*u[4]*u[0]/mass;          
    f[3] = -g - q*u[4]*u[1]/mass;
    f[4] = u[0]**2 + u[1]**2 - l**2;
    return f

# Матрица системы D

In [7]:
def D():
    D = zeros((5,5))
    
    for i in range(4):           
        D[i,i] = 1.
    return D

# Матрица Якоби

In [8]:
def f_u(u,g,mass,l):
    f_u = zeros((5,5))

    f_u[0,2] = 1
    f_u[1,3] = 1
    f_u[2,0] = -q*u[4]/mass       
    f_u[2,4] = -q*u[0]/mass
    f_u[3,1] = -q*u[4]/mass
    f_u[3,4] = -q*u[1]/mass
    f_u[4,0] = 2*u[0]
    f_u[4,1] = 2*u[1]
    return f_u




# Проход схемы Ваннера-Розенброка

In [9]:
u = zeros((M + 1,5))
u[0,:] = [x_0, y_0, v_x_0, v_y_0, mass*g] 
for m in range(M):
    g=gravity(t[m])
    w_1 = linalg.solve(D() - alpha*tau*f_u(u[m],g,mass,L), f(u[m],g,mass,L))
    u[m + 1] = u[m] + tau*w_1.real


# Отрисовка траекторий $x(t)$ и $y(t)$

In [10]:
plot(t, u.T[0], label='$x(t)$')
plot(t, u.T[1], label='$y(t)$')
xlabel('t')
legend()
show()

# Отрисовка $f(t)=\sqrt{x^2(t)+y^2(t)}$ в осях $(t, f)$

In [11]:
figure()
plot(t, u.T[0]**2+u.T[1]**2, label='$\sqrt{x^2(t)+y^2(t)}$')
xlabel('t')
legend(loc='upper right')
show()

# Анимация движения маятника 

In [15]:
fig = figure()
ax = axes(xlim=(-5*1.62,5*1.62), ylim=(-7.5,3.5))
plot(0,0,'or', color="red", marker='o', markersize=7)
plot((-2,2),(0,0),'-')
rod, = ax.plot([], [], color="blue", lw=3)
bob, = ax.plot([], [], color="blue", marker='o', markersize=7)

def init():
    rod.set_data([], [])
    bob.set_data([], [])
    return rod,bob,                                 

def animate(i):
    rod.set_data((0,u[i,0]),(0,u[i,1]))
    bob.set_data(u[i,0],u[i,1])
    return rod,bob,
anim = FuncAnimation(fig, animate, init_func=init,frames = M + 1, interval = 15, blit=True)
